# Исключения
## Домашняя работа

### Вопросы по лекциям.

Как поймать вообще все ошибки, которые могут произойти?

**Ответ:** ключевым словом `except` без явного указания типа исключения
```
try:
    # Buggy code
except:
    # Any possible error could be handled here
```

Сколько раз подряд можно указывать except?

**Ответ:** По одному для каждого типа ожидаемой ошибки
```
try:
    # buggy code
except TerribleError:
    # ...
except TerrifiyngError:
    # ...
except:
    # all other errors
```

Вы хотите с помощью print вывести название ошибки в консоль, как это сделать?

**Ответ:** использовать объект исключения
```
try:
    # buggy code
except Exception as exc:
    print(f"Got a {exc} error!")
```

Вы хотите с помощью print вывести параметры ошибки в консоль, как это сделать?

**Ответ:** аналогично предыдущему вопросу, но используя параметр `args` объекта ошибки:
```
print(f"Got a {exc} error with args: {exc.args}!")
```

Что такое DeprecationWarning?

**Ответ:**
предупреждение о том, что используемая часть API является устаревшей и, вероятно, будет удалена из пакета в следующих релизах.

### Разминочные задания. 
<br>

Вам даны две функции. Поисследуйте, какие ошибки могут возникнуть при их реализации. Обработайте эти ошибки.

In [13]:
def div():
    
    for i in range(2):
        try:
            x = int(input("enter a number: "))
            y = int(input("enter another number: "))
            print(x, '/', y, '=', x/y)
        except ValueError as exc:
            print(f"Only integers numbers are allowed! Got: {exc}")
        except ZeroDivisionError as exc:
            print(f"Got '{exc}' error! Make sure that the second number is not 0!")
        except Exception as exc:
            print(f"Got unexpected error: {exc}")

div()

enter a number: 1
enter another number: 0
Got division by zero! Make sure that the second number is not 0!
enter a number: 1
enter another number: 2
1 / 2 = 0.5


In [29]:
def sumOfPairs(L1, L2):
    sum = 0
    sumOfPairs = []
    for i in range(len(L1)):
        sumOfPairs.append(None)
        try:
            sumOfPairs[-1] = L1[i]+L2[i]
        except IndexError as exc:
            print(f"Got '{exc}' error. Do lists have an equal length?")
        except TypeError as exc:
            print(f"Got '{exc}' error at position {i}")
        except Exception as exc:
            print(f"Got unexpected error: '{exc}'")

    print("sumOfPairs = ", sumOfPairs)
    
    
sumOfPairs([1, 2, 3], ["1", "2"])

Got 'unsupported operand type(s) for +: 'int' and 'str'' error at position 0
Got 'unsupported operand type(s) for +: 'int' and 'str'' error at position 1
Got 'list index out of range' error. Do lists have an equal length?
sumOfPairs =  [None, None, None]


### Задание 1.

Есть файл с протоколом регистраций пользователей на сайте (registrations.txt).<br>
Каждая строка содержит информацию о имени, электронной почте и возрасте человека. <br><br>


Надо проверить данные из файла, для каждой строки:
 - присутсвуют все три поля
 - поле имени содержит только буквы
 - поле email содержит @ и .
 - поле возраст является числом от 10 до 99<br>

В результате проверки нужно сформировать два файла
 - registrations_good.log для правильных данных, записывать строки как есть
- registrations_bad.log для ошибочных, записывать строку и вид ошибки.<br>

Для валидации строки данных написать метод, который может выкидывать исключения:
 - НЕ присутсвуют все три поля: ValueError
 - поле имени содержит НЕ только буквы: NotNameError (кастомное исключение)
 - поле email НЕ содержит @ и .(точку): NotEmailError (кастомное исключение)
 - поле возраст НЕ является числом от 10 до 99: ValueError
Вызов метода обернуть в try-except.

In [53]:
class NotNameError(Exception):
    def __init__(self, bad_name: str):
        self.args = (bad_name,)
        self.message = f"Invalid name '{bad_name}'"

    def __str__(self) -> str:
        return self.message
        
class NotEmailError(Exception):
    def __init__(self, bad_mail: str):
        self.args = (bad_mail,)
        self.message = f"Invalid e-mail '{bad_mail}'"

    def __str__(self) -> str:
        return self.message
        
def check_email(email: str) -> bool:
    try:
        name, domain = email.split("@")
        company, area = domain.split(".")
        return True
    except ValueError:
        return False
        
def check_record(row: str):
    try:
        name, email, age_str = row.split(" ")
    except ValueError as exc:
        raise ValueError("Not all required data provided!")
        
    if not name.isalpha():
        raise NotNameError(name)
    if not check_email(email):
        raise NotEmailError(email)
    
    age = int(age_str)
    if age > 99 or age < 10:
        raise ValueError("Age must be between 10 and 99!")

good_records = open("registrations_good.log", "w")
bad_records = open("registrations_bad.log", "w")

with open("hw_9_registrations.txt", "r", encoding="UTF-8") as data:
    for row in data:
        row = row.strip(" \n")
        try:
            check_record(row)
            print(row, file=good_records)
        except Exception as exc:
            print(f"{row} -- {exc}", file=bad_records)
            
good_records.close()
bad_records.close()
